# Heart Disease Classification - SVC

## 1. Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## 2. Data Preprocessing

In [2]:
# Load dataframe (df)
df = pd.read_csv('heart.csv')
X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values

print(X.shape, y.shape)

(303, 13) (303,)


In [3]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [4]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
target        int64
dtype: object

In [5]:
# Check for missing values
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [6]:
# Describe dataframe
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [7]:
# Split dataframe (df) into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=0)

In [8]:
# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 3. Define SVC model

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

In [10]:
# Accuracy scoring
svc_model = SVC(C=0.28, kernel='rbf', gamma=0.08, random_state=0)

# ROC_AUC scoring
# clf = SVC(C=0.75, kernel='linear', random_state=0)
svc_model.fit(X_train, y_train)


SVC(C=0.28, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.08, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

## 4. Evaluate SVC Model

In [11]:
# Predict test set results
svc_pred = svc_model.predict(X_test)

# Create the confusion matrix
# TN = cm[0][0]  # True Negative
# FP = cm[0][1]  # False Positive
# FN = cm[1][0]  # False Negative
# TP = cm[1][1]  # True Positive
print('Confusion Matrix: ')
cm = confusion_matrix(y_test, svc_pred)
print(cm)
print('')
print("Val. Accuracy: %.3f%% " % (accuracy_score(y_test, svc_pred) * 100))
print("ROC AUC: %.3f%% " % (roc_auc_score(y_test, svc_pred) * 100))
print('')

# accuracy = (TP + TN)/(TN + FP + FN + TP)
# precision = TP/(TP + FP)
# recall = TP/(TP + FN)
# f1_score = (2 * precision * recall)/(precision + recall)
print('Classification Report: ')
print(classification_report(y_test, svc_pred))

Confusion Matrix: 
[[20  7]
 [ 2 32]]

Val. Accuracy: 85.246% 
ROC AUC: 84.096% 

Classification Report: 
              precision    recall  f1-score   support

           0       0.91      0.74      0.82        27
           1       0.82      0.94      0.88        34

    accuracy                           0.85        61
   macro avg       0.86      0.84      0.85        61
weighted avg       0.86      0.85      0.85        61



In [12]:
# Apply k-Fold Cross Validation on the training set
cv = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 0)
scores = cross_val_score(svc_model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# scores = cross_val_score(svc_models, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)

# Force scores to be positive
scores = abs(scores)
print("Mean Accuracy: %.3f%%" % (np.mean(scores) * 100))
print("Standard Deviation: %.3f%%" % np.std(scores))
print("Min Accuracy: %.3f%%" % (np.min(scores) * 100))
print("Max Accuracy: %.3f%%" % (np.max(scores) * 100))

Mean Accuracy: 83.461%
Standard Deviation: 0.056%
Min Accuracy: 70.833%
Max Accuracy: 100.000%


<table border = "1">
  <thead>
    <th colspan = "3">SVC Cross-Validation Results</th>
  </thead>
  <tbody>
    <tr>
      <td><b></b></td>
      <td><b>Accuracy</b></td>
      <td><b>ROC_AUC</b></td>
    </tr>
    <tr>
      <td><b>Mean</b></td>
      <td>83.461%</td>
      <td>89.466%</td>
    </tr>
    <tr>
      <td><b>Std.</b></td>
      <td>0.056%</td>
      <td>0.044%</td>
    </tr>
    <tr>
      <td><b>Min</b></td>
      <td>70.833%</td>
      <td>81.119%</td>
    </tr>
    <tr>
      <td><b>Max</b></td>
      <td>100.000%</td>
      <td>95.804%</td>
    </tr>
  </tbody>
</table>

In [13]:
# Apply Grid Search for parameter tuning

# svc_params = [{'C': [0.25, 0.3, 0.4, 0.5, 0.6, 0.75, 0.8, 0.9, 1], 
#                'kernel': ['linear'],
#                'random_state': [0, 1]},
#               {'C': [0.25, 0.275, 0.3, 0.4, 0.5, 0.6, 0.75, 0.8, 0.9, 1], 
#                'kernel': ['rbf'],
#                'gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#                'random_state': [0, 1]}]

# Accuracy scoring
svc_params = [{'C': [0.27, 0.28, 0.29], 
               'kernel': ['rbf'],
               'gamma': [0.07, 0.08, 0.09],
               'random_state': [0, 1]}]

# ROC_AUC scoring
# svc_params = [{'C': [0.74, 0.75, 0.76], 
#                'kernel': ['linear'],
#                'random_state': [0]}]

svc_grid = GridSearchCV(svc_model, svc_params, scoring='accuracy', cv=cv, n_jobs=-1)
# grid = GridSearchCV(svc_model, svc_params, scoring='roc_auc', cv=cv, n_jobs=-1)
results = svc_grid.fit(X_train, y_train)

# Summarize results
best_accuracy = svc_grid.best_score_
best_parameters = svc_grid.best_params_
print("Best Accuracy: %.3f%%" % (best_accuracy * 100))
print("Best Parameters: ", best_parameters)


Best Accuracy: 83.461%
Best Parameters:  {'C': 0.28, 'gamma': 0.08, 'kernel': 'rbf', 'random_state': 0}


<table border = "1">
  <thead>
    <th colspan = "3">SVC Grid Search Results</th>
  </thead>
  <tbody>
    <tr>
      <td><b></b></td>
      <td><b>Accuracy</b></td>
      <td><b>ROC_AUC</b></td>
    </tr>
    <tr>
      <td><b>Best Score</b></td>
      <td>83.461%</td>
      <td>89.466%</td>
    </tr>
    <tr>
      <td colspan="3"><b>Best Parameters:</b></td>
    </tr>
    <tr>
      <td><b>kernel</b></td>
      <td>rbf</td>
      <td>linear</td>
    </tr>
    <tr>
      <td><b>C</b></td>
      <td>0.28</td>
      <td>0.75</td>
    </tr>
    <tr>
      <td><b>gamma</b></td>
      <td>0.08</td>
      <td>N/A</td>
    </tr>
    <tr>
      <td><b>random_state</b></td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>


## 5. Final Scores

<table border = "1">
  <thead>
    <th colspan = "3">SVC Model Final Scores</th>
  </thead>
    <tr>
      <td><b></b></td>
      <td><b>Pred.</b></td>
      <td><b>Val.</b></td>
    </tr>
    <tr>
      <td><b>Accuracy (%):</b></td>
      <td>83.461</td>
      <td>85.246</td>
    </tr>
    <tr>
      <td><b>ROC AUC (%):</b></td>
      <td>89.466</td>
      <td>84.096</td>
    </tr>
  </tbody>
</table>